In [1]:
# allow the notebook to access the parent directory so we can import the other modules
# https://stackoverflow.com/a/35273613
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

# Data Preparation
-----

### Constants and Folder Paths

In [2]:
import os
dataset_folder_path = os.path.join("..", "files", "dataset")

### Load Data and Split into *Test*, *Train/Valid*

In [3]:
from data.DataSet import DataSet
dataset = DataSet()
dataset.load(dataset_folder_path, test_set_percentage=0, validation_set_percentage=0)

In [4]:
print(len(dataset.train_data))
print(len(dataset.test_data))

3600
0


### Data Preprocessing

In [5]:
from utils.preprocessing import *
from functools import partial
dataset.apply(apply_mean_centering)
dataset.apply(apply_unit_distance_normalization)
dataset.expand(reverse_digit_sequence)

In [6]:
print(len(dataset.train_data))
print(len(dataset.test_data))

7200
0


----------
## Test different sample rates and cross validate then compare the results to determine the optimal sampling rate to use
----------

In [9]:
NUM_SAMPLES_TO_TRY = [10, 25, 50, 75, 100, 125, 150, 200, 250, 300]
N_FOLDS = 10
PARAM_NUM_EPOCHS = 30
PARAM_BATCH_SIZE = 300

In [10]:
import numpy as np
import pandas as pd
from utils.evaluation import cross_validate_model
from models.regularized_deep_gru import NaiveRegularizedDeepGRU

results = {}

for num_samples in NUM_SAMPLES_TO_TRY:
    print("Evaluating Spline interpolation using %d samples\n-------------------------------" % num_samples)
    # setup copy of data and evaluate its spline with the currently selected number of samples
    data = dataset.copy()
    data.apply(partial(spline_interpolate_and_resample, num_samples=num_samples))
    x = np.array(data.train_data)
    y = np.array(data.train_labels)
    # setup the model
    mymodel = NaiveRegularizedDeepGRU(x.shape[1:])
    mymodel.batch_size = PARAM_BATCH_SIZE
    mymodel.num_epochs = PARAM_NUM_EPOCHS
    # run cross validation
    scores = cross_validate_model(x, y, mymodel, N_FOLDS)
    results[sample_rate] = scores
    
results = pd.DataFrame([[key, res[0], res[1]] for key,res in list(results.items())], columns=["Number of Samples", "Categorical Accuracy", "Std Deviation"])
results

Evaluating Spline interpolation using %d samples
------------------------------- 10
Train on 6480 samples, validate on 720 samples
Epoch 1/30
6480/6480 [==============================] - 2s 277us/step - loss: 1.9061 - categorical_accuracy: 0.3457 - val_loss: 2.0463 - val_categorical_accuracy: 0.2847
Epoch 2/30
6480/6480 [==============================] - 1s 147us/step - loss: 1.0179 - categorical_accuracy: 0.6520 - val_loss: 0.6752 - val_categorical_accuracy: 0.7639
Epoch 3/30
2100/6480 [========>.....................] - ETA: 0s - loss: 0.5591 - categorical_accuracy: 0.8052

KeyboardInterrupt: 